In [1]:
# if you want to use cuda, you can specify the ID of the device
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'
# and set the use_cuda parameter to True
use_cuda = True # otherwise, set it to False

In [2]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

In [3]:
from ru_detoxification_evaluation import load_model

## Reading input dataset and results of model inference

In [5]:
# We suppose that the input dataset is a dataframe with original toxic sentences 
# and/or with neutral refenrences
df = pd.read_csv('../data/input/dev.tsv', sep='\t')
df = df.fillna('')

In [7]:
df.head(2)

,toxic_comment,neutral_comment1,neutral_comment2,neutral_comment3
0,пиздеж! температуры горения хватит чтобы её ра...,Враньё! Температуры горения хватит чтобы ее ра...,"неправда,температуры горения хватит чтобы расп...",Враньё! Температуры горения хватит на чтобы её...
1,а ты чмо там был.ты вообще служил.гандон,А ты там был? Ты вообще служил?,,


In [8]:
toxic_inputs = df['toxic_comment'].tolist()

neutral_references = []
for index, row in df.iterrows():
    neutral_references.append([row['neutral_comment1'], row['neutral_comment2'], row['neutral_comment3']])

In [10]:
# We suppose that the model outputs are saved as .txt file seperated with '\n'
with open('../data/output/t5_base_10000_dev.txt', 'r') as file:
    preds = file.readlines()
preds = [sentence.strip() for sentence in preds]

## Style Transfer Accuracy (STA)

In [11]:
from ru_detoxification_metrics import evaluate_style

In [12]:
style_model, style_tokenizer = load_model('SkolkovoInstitute/russian_toxicity_classifier', use_cuda=use_cuda)

In [13]:
accuracy = evaluate_style(
    model = style_model,
    tokenizer = style_tokenizer,
    texts = preds,
    target_label=0,  # 1 is toxic, 0 is neutral
    batch_size=32, 
    verbose=True
)

In [14]:
print(f'Style transfer accuracy (STA):  {np.mean(accuracy)}')

Style transfer accuracy (STA):  0.738360583782196


## Meaning Preservation Score (SIM)

In [15]:
from ru_detoxification_metrics import evaluate_cosine_similarity

In [16]:
meaning_model, meaning_tokenizer = load_model('cointegrated/LaBSE-en-ru', use_cuda=use_cuda, model_class=AutoModel)

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
similarity = evaluate_cosine_similarity(
    model = meaning_model,
    tokenizer = meaning_tokenizer,
    original_texts = toxic_inputs,
    rewritten_texts = preds,
    batch_size=32,
    verbose=True,
    )

In [18]:
print(f'Meaning preservation (SIM):  {np.mean(similarity)}')

Meaning preservation (SIM):  0.7641064524650574


## Fluency score (FL)

In [19]:
from ru_detoxification_metrics import evaluate_cola_relative

In [20]:
cola_model, cola_tolenizer = load_model('SkolkovoInstitute/rubert-base-corruption-detector', use_cuda=use_cuda)

In [21]:
fluency = evaluate_cola_relative(
    model = cola_model,
    tokenizer = cola_tolenizer,
    original_texts = toxic_inputs,
    rewritten_texts = preds,
    target_label=1,
    batch_size=32,
    verbose=True
)

In [22]:
print(f'Fluency score (FL):  {np.mean(fluency)}')

Fluency score (FL):  0.7833276391029358


## Joint score (J)

In [23]:
joint = accuracy * similarity * fluency

In [24]:
print(f'Joint score (J):   {np.mean(joint)}')

Joint score (J):   0.45232293009757996


## ChrF1 with references

In [25]:
from nltk.translate.chrf_score import corpus_chrf

In [26]:
corpus_chrf(neutral_references, preds)

0.5666207576899206